In [7]:
# Instal dependency
# !pip install open_clip_torch

In [8]:
# Import module
import os
import open_clip
import glob
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm

# Parse data path

In [9]:
keyframes_dir = '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe'
all_keyframe_paths = dict()
for part in sorted(os.listdir(keyframes_dir)):
    data_part = part.split('_')[-1] # L01, L02 for ex
    all_keyframe_paths[data_part] =  dict()

for data_part in sorted(all_keyframe_paths.keys()):
    data_part_path = f'{keyframes_dir}/{data_part}'
    video_dirs = sorted(os.listdir(data_part_path))
    video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
    for video_id, video_dir in zip(video_ids, video_dirs):
        keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
        all_keyframe_paths[data_part][video_id] = keyframe_paths

print(all_keyframe_paths)

{'Video0': {'L00': ['/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe/Video0/L00/keyframe_10_44.611233.jpg', '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe/Video0/L00/keyframe_11_52.952900.jpg', '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe/Video0/L00/keyframe_12_58.825433.jpg', '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe/Video0/L00/keyframe_13_67.167100.jpg', '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe/Video0/L00/keyframe_14_73.573500.jpg', '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe/Video0/L00/keyframe_15_77.811067.jpg', '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe/Video0/L00/keyframe_16_86.152733.jpg', '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe/Video0/L00/keyframe_17_88.988900.jpg', '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe/Video0/L00/keyframe_

# Model

In [10]:
##### Load Model #####
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', device=device, pretrained='datacomp_xl_s13b_b90k')

cpu


In [11]:
bs = 4
save_dir = './CLIPv2_features'
if not os.path.exists(save_dir):
  os.mkdir(save_dir)

for key, video_keyframe_paths in tqdm(all_keyframe_paths.items()):
    video_ids = sorted(video_keyframe_paths.keys())
    
    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))
    
    for video_id in tqdm(video_ids):
        video_feats = []
        video_keyframe_path = video_keyframe_paths[video_id]
        for i in range(0, len(video_keyframe_path), bs):
            # Support batchsize inferencing
            images = []
            image_paths = video_keyframe_path[i:i+bs]
            for image_path in image_paths:
                image = preprocess(Image.open(image_path)).unsqueeze(0)
                images.append(image)
            images = torch.cat(images).to(device)

            with torch.no_grad(), torch.cuda.amp.autocast():
                image_feats = model.encode_image(images)
            image_feats /= image_feats.norm(dim=-1, keepdim=True)

            for b in range(image_feats.shape[0]):
                video_feats.append(image_feats[b].detach().cpu().numpy().astype(np.float32).flatten())
        
        np.save(f'{save_dir}/{key}/{video_id}.npy', video_feats)

  0%|          | 0/3 [00:00<?, ?it/s]/var/folders/fw/f3018xmn4d91q9730vjl_p_r0000gn/T/ipykernel_44417/1485413450.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
100%|██████████| 3/3 [00:18<00:00,  6.17s/it]
